# Machine Learning with Dynamic Time Patterns for Algorithmic Trading

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("usd_dkk_5min.csv", parse_dates=["Time"])

# Sort the index to ensure monotonicity
df.reset_index(drop=True, inplace=True)
df.set_index("Time", inplace=True)
df = df.sort_index()


# Define the date range
start_date = "2024.01.01.17:15"
end_date = "2024.01.02.21:25"

# Filter to the first 10 days of 2024
df = df[start_date:end_date].copy()
print(df.tail())


                        Open     High      Low    Close
Time                                                   
2024-01-02 21:05:00  6.80745  6.80842  6.80739  6.80832
2024-01-02 21:10:00  6.80832  6.80916  6.80830  6.80883
2024-01-02 21:15:00  6.80882  6.81001  6.80851  6.80858
2024-01-02 21:20:00  6.80858  6.80945  6.80844  6.80932
2024-01-02 21:25:00  6.80933  6.80976  6.80896  6.80960


In [3]:
# Calculate log returns
df["returns"] = 1+np.log(df["Close"] / df["Close"].shift(1))
df.dropna(inplace=True)

In [4]:
from tslearn.metrics import dtw
from sklearn.neighbors import NearestNeighbors
import numpy as np
from fastdtw import fastdtw


def knn_dtw_forecast(series, window_size=150, embed_size=10, k=7):
    predictions = []
    
    # FastDTW metric for kNN
    def dtw_metric(x, y):
        x_norm = (x - x.mean()) / x.std()
        y_norm = (y - y.mean()) / y.std()
        return fastdtw(x_norm, y_norm)[0]  # Returns only distance

    # Main loop
    for i in range(window_size + embed_size, len(series)):
        query = series.iloc[i - embed_size:i].values
        history = series.iloc[:i - embed_size].values

        if len(history) < window_size:
            predictions.append(0)
            continue

        # Create rolling window of historical embeddings
        start_index = max(0, len(history) - window_size)
        historical_embeddings = np.array([history[j:j + embed_size] for j in range(start_index, len(history) - embed_size + 1)])

        if query.std() == 0 or np.any(historical_embeddings.std(axis=1) == 0):
            predictions.append(0)
            continue

        # Parallelized kNN search
        nbrs = NearestNeighbors(n_neighbors=k, metric=dtw_metric, n_jobs=-1).fit(historical_embeddings)
        dist, indices = nbrs.kneighbors([query])

        # Predict next return using weighted average
        next_returns = [series.iloc[start_index + idx + embed_size] for idx in indices[0]]
        weights = 1 / (dist[0] + 1e-6)  # Avoid div by zero
        prediction = np.average(next_returns, weights=weights)

        predictions.append(prediction)
        
    # Padding at the start
    return [0] * (window_size + embed_size) + predictions

# Run optimized forecast
df["dtw_pred"] = knn_dtw_forecast(df["returns"])


In [5]:
print(df["returns"])
print(df["dtw_pred"])

Time
2024-01-01 17:20:00    1.000080
2024-01-01 17:25:00    1.000064
2024-01-01 17:30:00    0.999961
2024-01-01 17:35:00    1.000164
2024-01-01 17:40:00    0.999978
                         ...   
2024-01-02 21:05:00    1.000126
2024-01-02 21:10:00    1.000075
2024-01-02 21:15:00    0.999963
2024-01-02 21:20:00    1.000109
2024-01-02 21:25:00    1.000041
Name: returns, Length: 338, dtype: float64
Time
2024-01-01 17:20:00    0.000000
2024-01-01 17:25:00    0.000000
2024-01-01 17:30:00    0.000000
2024-01-01 17:35:00    0.000000
2024-01-01 17:40:00    0.000000
                         ...   
2024-01-02 21:05:00    0.999915
2024-01-02 21:10:00    1.000086
2024-01-02 21:15:00    0.999979
2024-01-02 21:20:00    0.999983
2024-01-02 21:25:00    1.000022
Name: dtw_pred, Length: 338, dtype: float64


In [6]:
def strategy(df,  buy_threshold=1.00002, sell_threshold=0.99998):
    df["position"] = 0  # 1 for long, -1 for short, 0 for neutral

    for i in range(160, len(df) - 1):

        # Determine position based on prediction
        if df['dtw_pred'].iloc[i] > buy_threshold:
            df.loc[df.index[i], 'position'] = 1

        elif df['dtw_pred'].iloc[i] < sell_threshold:
            df.loc[df.index[i], 'position'] = -1
        else:
            df.loc[df.index[i], 'position'] = 0


    return df
strategy(df)

,Open,High,Low,Close,returns,dtw_pred,position
Time,,,,,,,
2024-01-01 17:20:00,6.74681,6.74790,6.74681,6.74740,1.000080,0.000000,0
2024-01-01 17:25:00,6.74735,6.74791,6.74735,6.74783,1.000064,0.000000,0
2024-01-01 17:30:00,6.74753,6.74792,6.74751,6.74757,0.999961,0.000000,0
2024-01-01 17:35:00,6.74757,6.74868,6.74743,6.74868,1.000164,0.000000,0
2024-01-01 17:40:00,6.74868,6.74904,6.74838,6.74853,0.999978,0.000000,0
...,...,...,...,...,...,...,...
2024-01-02 21:05:00,6.80745,6.80842,6.80739,6.80832,1.000126,0.999915,-1
2024-01-02 21:10:00,6.80832,6.80916,6.80830,6.80883,1.000075,1.000086,1
2024-01-02 21:15:00,6.80882,6.81001,6.80851,6.80858,0.999963,0.999979,-1


In [7]:
print(df["position"])

Time
2024-01-01 17:20:00    0
2024-01-01 17:25:00    0
2024-01-01 17:30:00    0
2024-01-01 17:35:00    0
2024-01-01 17:40:00    0
                      ..
2024-01-02 21:05:00   -1
2024-01-02 21:10:00    1
2024-01-02 21:15:00   -1
2024-01-02 21:20:00    0
2024-01-02 21:25:00    0
Name: position, Length: 338, dtype: int64


In [8]:
print(df.tail(10))

                        Open     High      Low    Close   returns  dtw_pred  \
Time                                                                          
2024-01-02 20:40:00  6.80829  6.80829  6.80657  6.80657  0.999765  1.000008   
2024-01-02 20:45:00  6.80647  6.80752  6.80633  6.80707  1.000073  0.999991   
2024-01-02 20:50:00  6.80707  6.80768  6.80694  6.80754  1.000069  0.999942   
2024-01-02 20:55:00  6.80760  6.80877  6.80741  6.80847  1.000137  0.999947   
2024-01-02 21:00:00  6.80847  6.80943  6.80744  6.80746  0.999852  1.000006   
2024-01-02 21:05:00  6.80745  6.80842  6.80739  6.80832  1.000126  0.999915   
2024-01-02 21:10:00  6.80832  6.80916  6.80830  6.80883  1.000075  1.000086   
2024-01-02 21:15:00  6.80882  6.81001  6.80851  6.80858  0.999963  0.999979   
2024-01-02 21:20:00  6.80858  6.80945  6.80844  6.80932  1.000109  0.999983   
2024-01-02 21:25:00  6.80933  6.80976  6.80896  6.80960  1.000041  1.000022   

                     position  
Time               

In [9]:
df.reset_index(drop=True, inplace=True)

def backtest(df, starting_balance, trade_risk):
    df["balance"] = np.nan
    df.loc[160, "balance"] = starting_balance
    df["trade_status"] = 0
    df["trade_size"]= np.nan
    df["trade_pnl"] = np.nan
    df["trade_success"] = np.nan

    long_trades = 0
    short_trades = 0

    
    for i in range(1, len(df)-1):
        df.loc[i, "balance"] = df.loc[i-1, "balance"]
        
        if df['position'].iloc[i] == 1 :
            df.loc[i, "trade_status"] = 1
            entry_price = df['Close'].iloc[i]
            exit_price = df['Close'].iloc[i+1]
            trade_size = df["balance"].iloc[i-1] * trade_risk
            pnl = trade_size * (exit_price - entry_price) / entry_price
            df.loc[i, "trade_pnl"] = pnl
            long_trades += 1
            
            if pnl > 0:
                df.loc[i, "trade_success"] = 1
            else:
                df.loc[i, "trade_success"] = 0


            df.loc[i, "balance"] += pnl


        elif df['position'].iloc[i] == -1:
            df.loc[i, "trade_status"] = -1
            entry_price = df['Close'].iloc[i]
            exit_price = df['Close'].iloc[i+1]
            trade_size = df["balance"].iloc[i-1] * trade_risk
            pnl = trade_size * (entry_price - exit_price) / entry_price
            df.loc[i, "trade_pnl"] = pnl
            short_trades += 1

            if pnl > 0:
                df.loc[i, "trade_success"] = 1
            else:
                df.loc[i, "trade_success"] = 0

            df.loc[i, "balance"] += pnl

        


    
    total_trades = long_trades + short_trades

    return df, long_trades, short_trades, total_trades


backtest(df, 100000, 0.02)



(        Open     High      Low    Close   returns  dtw_pred  position  \
 0    6.74681  6.74790  6.74681  6.74740  1.000080  0.000000         0   
 1    6.74735  6.74791  6.74735  6.74783  1.000064  0.000000         0   
 2    6.74753  6.74792  6.74751  6.74757  0.999961  0.000000         0   
 3    6.74757  6.74868  6.74743  6.74868  1.000164  0.000000         0   
 4    6.74868  6.74904  6.74838  6.74853  0.999978  0.000000         0   
 ..       ...      ...      ...      ...       ...       ...       ...   
 333  6.80745  6.80842  6.80739  6.80832  1.000126  0.999915        -1   
 334  6.80832  6.80916  6.80830  6.80883  1.000075  1.000086         1   
 335  6.80882  6.81001  6.80851  6.80858  0.999963  0.999979        -1   
 336  6.80858  6.80945  6.80844  6.80932  1.000109  0.999983         0   
 337  6.80933  6.80976  6.80896  6.80960  1.000041  1.000022         0   
 
      balance  trade_status  trade_size  trade_pnl  trade_success  
 0        NaN             0         NaN   

In [10]:
print(df.tail(10))


        Open     High      Low    Close   returns  dtw_pred  position  \
328  6.80829  6.80829  6.80657  6.80657  0.999765  1.000008         0   
329  6.80647  6.80752  6.80633  6.80707  1.000073  0.999991         0   
330  6.80707  6.80768  6.80694  6.80754  1.000069  0.999942        -1   
331  6.80760  6.80877  6.80741  6.80847  1.000137  0.999947        -1   
332  6.80847  6.80943  6.80744  6.80746  0.999852  1.000006         0   
333  6.80745  6.80842  6.80739  6.80832  1.000126  0.999915        -1   
334  6.80832  6.80916  6.80830  6.80883  1.000075  1.000086         1   
335  6.80882  6.81001  6.80851  6.80858  0.999963  0.999979        -1   
336  6.80858  6.80945  6.80844  6.80932  1.000109  0.999983         0   
337  6.80933  6.80976  6.80896  6.80960  1.000041  1.000022         0   

     balance  trade_status  trade_size  trade_pnl  trade_success  
328      NaN             0         NaN        NaN            NaN  
329      NaN             0         NaN        NaN            N